In [95]:
# Load data
with open("../data/day19test.txt", "r") as f:
    input = f.read()

parts = input.split("\n\n")
rules = parts[0]
msgs = parts[1]

# real data is not sorted!
rule_list = [rule[3:] for rule in rules.split("\n")]
rule_list

['4 1 5', '2 3 | 3 2', '4 4 | 5 5', '4 5 | 5 4', '"a"', '"b"']

In [176]:
# Puzzle 1
# Questions:
# - build list of allowed options? might be too much
# Options:
# - recursion function (see below): turns out to be difficult
# - as tree, possibly object-oriented? but then how to follow the tree when testing the messages?
import re
regex_rule = re.compile(r"(\d+ \d+ \| \d+ \d+)|((?:\d+ ?)+)|\"([ab])\"")

# Recursion test, not correct
# rule_list should be known
rule = [""]*10
def parse(rule, k, n):
    """
    rule = ruleset to build
    k = 
    n = start rule
    """
    # do regex and named groups for main rules
    print(rule, rule_list[n], k)
    results = regex_rule.search(rule_list[n])
    groups = results.groups()

    if groups[0]: # 2 3 | 3 2
        parts = groups[0].split("|")
        #rule[k + 1] = rule[k]
        for rule_n in parts[0].strip().split(" "):
            rule = parse(rule, k, int(rule_n))
        k += 1 # split here
        for rule_n in parts[1].strip().split(" "):
            rule = parse(rule, k, int(rule_n))
    if groups[1]: # 4 1 5
        for rule_n in groups[1].strip().split(" "):
            rule = parse(rule, k, int(rule_n))
    if groups[2]: # a b
        if groups[2] == "a":
            rule[k] = rule[k] + "a"
        if groups[2] == "b":
            rule[k] = rule[k] + "b"
    return rule

parse(rule, 0, 0)

['', '', '', '', '', '', '', '', '', ''] 4 1 5 0
['', '', '', '', '', '', '', '', '', ''] "a" 0
['a', '', '', '', '', '', '', '', '', ''] 2 3 | 3 2 0
['a', '', '', '', '', '', '', '', '', ''] 4 4 | 5 5 0
['a', '', '', '', '', '', '', '', '', ''] "a" 0
['aa', '', '', '', '', '', '', '', '', ''] "a" 0
['aaa', '', '', '', '', '', '', '', '', ''] "b" 1
['aaa', 'b', '', '', '', '', '', '', '', ''] "b" 1
['aaa', 'bb', '', '', '', '', '', '', '', ''] 4 5 | 5 4 0
['aaa', 'bb', '', '', '', '', '', '', '', ''] "a" 0
['aaaa', 'bb', '', '', '', '', '', '', '', ''] "b" 0
['aaaab', 'bb', '', '', '', '', '', '', '', ''] "b" 1
['aaaab', 'bbb', '', '', '', '', '', '', '', ''] "a" 1
['aaaab', 'bbba', '', '', '', '', '', '', '', ''] 4 5 | 5 4 1
['aaaab', 'bbba', '', '', '', '', '', '', '', ''] "a" 1
['aaaab', 'bbbaa', '', '', '', '', '', '', '', ''] "b" 1
['aaaab', 'bbbaab', '', '', '', '', '', '', '', ''] "b" 2
['aaaab', 'bbbaab', 'b', '', '', '', '', '', '', ''] "a" 2
['aaaab', 'bbbaab', 'ba', '', '', 

['aaaabb', 'bbbaabaa', 'babb', '', '', '', '', '', '', '']

In [98]:
rule_list

['4 1 5', '2 3 | 3 2', '4 4 | 5 5', '4 5 | 5 4', '"a"', '"b"']

In [217]:
# Puzzle 1 with objects

# Not trees per se, but object-oriented, to easily copy objects.

# Parsing function:
#rule:
# if a/b: store value, drop doing, move one of todo to doing, stop if todo is empty
# if 1 2 3: put first into doing, rest into todo. 
# if 1 2 | 2 1: copy object, put both firsts into doing, rest into todo. call parsing functions of both

# to improve: inherit rule_list from parent

import re
regex_rule = re.compile(r"(\d+ \d+ \| \d+ \d+)|((?:\d+ ?)+)|\"([ab])\"")

class RuleSet:
    def __init__(self, rule_list):
        self.rule_list = rule_list
        self.rules = []

    def parse(self):
        """Create initial rule"""
        rule = Rule(self.rule_list, ruleset=self)
        self.rules.append(rule)

class Rule:
    def __init__(self, rule_list, todo=[], result="", ruleset=None, start_n=0):
        self.rule_list = rule_list
        self.start_n = start_n
        self.todo = todo
        self.result = result
        self.ruleset = ruleset

        self.parse(start_n)

    def parse(self, n=0):
        rule_to_parse = self.rule_list[n]
        groups = regex_rule.search(rule_to_parse).groups() # to do: named groups for main rules
        if groups[0]: # 2 3 | 3 2
            parts = groups[0].split("|")

            queue2 = [int(value) for value in parts[1].strip().split(" ")]
            todo = queue2 + self.todo
            doing = todo.pop(0)
            rule_new = Rule(self.rule_list, todo, self.result, self.ruleset, doing)
            self.ruleset.rules.append(rule_new)

            queue1 = [int(value) for value in parts[0].strip().split(" ")]
            self.todo += queue1
            self.parse(self.todo.pop(0))
            
        if groups[1]: # 4 1 5
            queue = [int(value) for value in groups[1].strip().split(" ")]
            self.todo = queue + self.todo
            self.parse(self.todo.pop(0))
        if groups[2]: # a b
            if groups[2] == "a":
                self.result += "a"
            if groups[2] == "b":
                self.result += "b"
            if self.todo:
                self.parse(self.todo.pop(0))
        return


rules = RuleSet(rule_list)
rules.parse()
[rule.result for rule in rules.rules]

['ababbb',
 'ababaa',
 'abbbab',
 'ababaa',
 'abbbba',
 'abbbab',
 'abbaaa',
 'abaaab']

In [205]:
rules.rules

In [182]:
x

[2, 3]